# Extraction of topics from Wikipedia pages

In [29]:
# https://github.com/taynaud/python-louvain

import IPython
import numpy as np
import networkx as nx
import requests
import pandas as pd
import csv
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import community
from fonctions import *


pd.set_option('mode.chained_assignment', None)

path = 'Results/EN_20180916_20180930/'

region = "fr"

In [2]:
### Importing the nodes as a DataFrame for easier manipulations ###

# dataFrame = dataFrame = pd.read_csv(path + 'nodes.csv', encoding='utf-8')

graph = nx.read_graphml(path+'graph.graphml')
dataFrame = pd.DataFrame.from_dict(dict(graph.nodes(data=True)), orient='index')
dataFrame['Id'] = dataFrame.index
dataFrame.rename(columns={'label': 'Label'}, inplace=True)
dataFrame['Label'] = dataFrame['Label'].astype('string')
dataFrame['Id'] = dataFrame['Id'].astype('int64')
# dataFrame.drop(columns=['size', 'r', 'g', 'b', 'x', 'y'], inplace=True)

degree = dict(nx.degree(graph, nbunch=None, weight=10))
partition = community.best_partition(graph, randomize=True, weight='Weight', resolution=1.5)
btw_cent = nx.betweenness_centrality(graph,normalized=False)
                           

dataFrame['Degree'] = pd.DataFrame.from_dict(degree, orient='index')[0]
dataFrame['modularity_class'] = pd.DataFrame.from_dict(partition, orient='index')[0]
dataFrame['betweenesscentrality'] = pd.DataFrame.from_dict(btw_cent, orient='index')[0]

add_graph_attribute(graph, dataFrame, 'Degree')
add_graph_attribute(graph, dataFrame, 'modularity_class')
add_graph_attribute(graph, dataFrame, 'betweenesscentrality')

dataFrame.sort_values(by = ['modularity_class'], inplace=True, ascending = [True])
dataFrame.sort_values(by = ['Id'], inplace=True, ascending = [True])
dataFrame.to_csv(path + 'nodes.csv', encoding='utf-8')

print(dataFrame.info())
display(dataFrame)

print("Number of clusters: ", dataFrame['modularity_class'].max())

<class 'pandas.core.frame.DataFrame'>
Index: 4834 entries, 61 to 12697556
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Label                 4834 non-null   string 
 1   size                  4834 non-null   float64
 2   r                     4834 non-null   int64  
 3   g                     4834 non-null   int64  
 4   b                     4834 non-null   int64  
 5   x                     4834 non-null   float64
 6   y                     4834 non-null   float64
 7   Id                    4834 non-null   int64  
 8   Degree                4834 non-null   int64  
 9   modularity_class      4834 non-null   int64  
 10  betweenesscentrality  4834 non-null   float64
dtypes: float64(4), int64(6), string(1)
memory usage: 453.2+ KB
None


,Label,size,r,g,b,x,y,Id,Degree,modularity_class,betweenesscentrality
61,Advanced_Micro_Devices,12.686567,192,192,192,9.234199,-16.520315,61,3,7,0.000000
103,Aton,20.746270,0,134,135,-10.841246,0.702035,103,9,13,2101.806936
206,Alfred_Nobel,11.343284,114,121,10,22.848211,24.862890,206,2,16,22046.317749
316,Lampe_à_incandescence_halogène,10.000000,114,121,10,-14.543655,-12.765390,316,1,16,0.000000
341,Amenhotep_III,31.492537,0,134,135,-10.848696,-14.589257,341,17,13,5114.899646
...,...,...,...,...,...,...,...,...,...,...,...
11983100,Thierry_Coste,23.432835,193,77,88,6.475411,-18.242767,11983100,11,8,46361.497269
11986417,Ugo_Humbert,14.029851,192,192,192,-8.752216,9.236148,11986417,4,7,0.000000
12470864,Équarrissage,10.000000,235,44,170,6.380493,19.105633,12470864,1,31,0.000000
12480619,Saison_9_de_Clem,12.686567,185,97,239,7.033388,-18.969147,12480619,3,4,0.000000


Number of clusters:  40


In [3]:
### Extract nodes labels from Wikipedia queries ###



# URL for the quieries
urls = "https://"+region+".wikipedia.org/w/api.php?action=query&prop=pageprops&format=json&pageids="


i=0
Nb_pages = len(dataFrame)


### Associate the 'Qid' value of Wikipedia pages in the DataFrame ###
### Give '-1' value if an error has occured during the query      ###

for pageId in np.int64(dataFrame['Id']):
    response = requests.get(urls + str(pageId)).json()
    try:
        Qid = (list(findkeys(response, 'wikibase_item'))[0])
        Title = (list(findkeys(response, 'title'))[0])
    except IndexError:
        dataFrame.loc[dataFrame['Id'] == pageId, 'Qid'] = '-1'
        i+=1
        print(i,"/", Nb_pages, dataFrame.loc[dataFrame['Id'] == pageId, 'Label'], "\t error")
    else:
        dataFrame.loc[dataFrame['Id'] == pageId, 'Qid'] = Qid
        dataFrame.loc[dataFrame['Id'] == pageId, 'Label'] = Title
        i+=1
        print(i,"/", Nb_pages, dataFrame.loc[dataFrame['Id'] == pageId, 'Label'].values[0])

# Clear the loading display
IPython.display.clear_output(wait=False)

        
# Save the DataFrame with their associated Qids
dataFrame.to_csv(path + 'pages_Qids.csv', encoding='utf-8')

dataFrame

,Label,size,r,g,b,x,y,Id,Degree,modularity_class,betweenesscentrality,Qid
61,Advanced Micro Devices,12.686567,192,192,192,9.234199,-16.520315,61,3,7,0.000000,Q128896
103,Aton,20.746270,0,134,135,-10.841246,0.702035,103,9,13,2101.806936,Q130227
206,Alfred Nobel,11.343284,114,121,10,22.848211,24.862890,206,2,16,22046.317749,Q23810
316,Lampe à incandescence halogène,10.000000,114,121,10,-14.543655,-12.765390,316,1,16,0.000000,Q1114190
341,Amenhotep III,31.492537,0,134,135,-10.848696,-14.589257,341,17,13,5114.899646,Q42606
...,...,...,...,...,...,...,...,...,...,...,...,...
11983100,Thierry Coste,23.432835,193,77,88,6.475411,-18.242767,11983100,11,8,46361.497269,Q56280447
11986417,Ugo Humbert,14.029851,192,192,192,-8.752216,9.236148,11986417,4,7,0.000000,Q54007306
12470864,Équarrissage,10.000000,235,44,170,6.380493,19.105633,12470864,1,31,0.000000,Q62649386
12480619,Saison 9 de Clem,12.686567,185,97,239,7.033388,-18.969147,12480619,3,4,0.000000,Q65223375


In [4]:
### Extracting topic pages from database API ###


dataFrame = pd.read_csv(path + 'pages_Qids.csv')

# Match_topic_API = pd.DataFrame(columns = ['Qid', 'topic', 'probability', 'page_id', 'page_title', 'modularity_class', 'wiki_db'])
Match_topic_API = pd.DataFrame()


                
### The API's URL from the topic is extracted ###         
urls = "http://86.119.25.229:5000/api/v1/wikidata/topic?qid="
threshold = '0.1'

    
tmps1=time.time()
    
a=0 
n = dataFrame.index[-1]
for pageIndex in dataFrame.index:
    pageQid = dataFrame.at[pageIndex, 'Qid']
    pageModularity = dataFrame.at[pageIndex, 'modularity_class']
    response = requests.get(urls + pageQid + "&threshold=" + threshold).json()
    scores = list(findkeys(response, 'score'))
    topics = list(findkeys(response, 'topic'))
    
    try:
        page_title = response['name']
    except KeyError:
        page_title = dataFrame.at[pageIndex, 'Label']
    for i in range(len(scores)):
        page = dataFrame.iloc[pageIndex]
        page['Topic'] = topics[i]
        page['Probability'] = scores[i]
        Match_topic_API = Match_topic_API.append(page, ignore_index=True)
    print(a, "/",n)
    a+=1
    
    
# Clear the loading display
IPython.display.clear_output(wait=False)
tmps2=time.time()
print("Execution time = ",(tmps2-tmps1), " s")

# Save the results 
Match_topic_API.drop(columns = ['Unnamed: 0'], inplace = True )
Match_topic_API.to_csv(path + 'pages_topic.csv', encoding='utf-8')

Match_topic_API

Execution time =  2187.063416481018  s


,Degree,Id,Label,Probability,Qid,Topic,b,betweenesscentrality,g,modularity_class,r,size,x,y
0,3.0,61.0,Advanced Micro Devices,0.909917,Q128896,History_and_Society.Business_and_economics,192.0,0.0,192.0,7.0,192.0,12.686567,9.234199,-16.520315
1,3.0,61.0,Advanced Micro Devices,0.835494,Q128896,STEM.STEM*,192.0,0.0,192.0,7.0,192.0,12.686567,9.234199,-16.520315
2,3.0,61.0,Advanced Micro Devices,0.782673,Q128896,STEM.Computing,192.0,0.0,192.0,7.0,192.0,12.686567,9.234199,-16.520315
3,3.0,61.0,Advanced Micro Devices,0.392347,Q128896,STEM.Technology,192.0,0.0,192.0,7.0,192.0,12.686567,9.234199,-16.520315
4,3.0,61.0,Advanced Micro Devices,0.242178,Q128896,Geography.Regions.Americas.North_America,192.0,0.0,192.0,7.0,192.0,12.686567,9.234199,-16.520315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22563,3.0,12480619.0,Saison 9 de Clem,0.938134,Q65223375,Culture.Media.Television,239.0,0.0,97.0,4.0,185.0,12.686567,7.033388,-18.969147
22564,3.0,12480619.0,Saison 9 de Clem,0.000000,Q65223375,Geography.Regions.Europe.Europe*,239.0,0.0,97.0,4.0,185.0,12.686567,7.033388,-18.969147
22565,3.0,12480619.0,Saison 9 de Clem,0.000000,Q65223375,Geography.Regions.Europe.Western_Europe,239.0,0.0,97.0,4.0,185.0,12.686567,7.033388,-18.969147
22566,1.0,12697556.0,Christophe Pélissier,0.569863,Q66309156,Culture.Biography.Biography*,183.0,0.0,168.0,0.0,0.0,10.000000,15.158192,17.343578


In [30]:
### Counting the number of topics per cluster and weighting them ###
### Finding the 3 most weighted topics per cluster with the higher degree page ###

df = pd.read_csv(path + 'pages_topic.csv', index_col = 'Unnamed: 0')


modularity_df = pd.DataFrame({'Main_Topics':[], 'Higher_Weighted_Article':[]})

df = count_topic(df)
df = weight_topic(df)


df.sort_values(by = ['modularity_class', 'Weight'], inplace=True, ascending = [True, False])


df.to_csv(path + 'pages_main_topic.csv', encoding='utf-8')

total_weight = df[['Weight', 'modularity_class']].groupby(['modularity_class']).sum()
total_weight = total_weight.reindex(pd.RangeIndex(total_weight.index.max() + 1)).fillna(value=0)

nb_topic = int( df['modularity_class'].max() ) + 1
for i in range(nb_topic):
    
    # If the cluster has no topic or only Biography/Geography. Ignored for other cases
    if (len(df.loc[df['modularity_class'] == i]) == 0):
        mod = {'Main_Topics':'Biography/Geography', 'Higher_Weighted_Article':''}
        modularity_df = modularity_df.append(mod, ignore_index=True)
        continue 
        
    
    df.loc[df['modularity_class'] == i, 'Total Weight'] = total_weight.iloc[i].values
    df['Ratio Weight'] = df['Weight'] / df['Total Weight']
    
    
    list_topic = []
    str_main_topics = ''
    idx_topic = 0
    max_idx_topic = len(df.loc[df['modularity_class'] == i])
    while ( (len(list_topic) < 3) and (idx_topic < max_idx_topic) ):
        str_topic = df.loc[df['modularity_class'] == i, 'Subtopic'].values[idx_topic]
        if (str_topic not in list_topic):
            list_topic.append(str_topic)
            str_main_topics +=  '/ ' + str_topic + ' (' + str((df.loc[(df['modularity_class'] == i) & (df['Subtopic'] == str_topic), 'Degree Ratio'].values[0]).round(decimals=2)) + ') '
        idx_topic += 1
    str_main_topics = str_main_topics[2:]
    
    
    
  
    page_id_max_weight = df.loc[df['modularity_class'] == i, 'Weight'].idxmax()
    str_weight = df.loc[page_id_max_weight, 'Label'] + ' (' + str(int(df.loc[page_id_max_weight, 'Weight'])) + ')'
        
    mod = {'Main_Topics':str_main_topics, 'Higher_Weighted_Article':str_weight}
    modularity_df = modularity_df.append(mod, ignore_index=True)


df.rename_axis("Index", inplace=True)    
display(df)    

display(modularity_df)


df.to_csv(path + 'pages_main_topic.csv', encoding='utf-8')

modularity_df.to_csv(path + 'topic_by_degree.csv', encoding='utf-8')

,Degree,Id,Label,Probability,Qid,Topic,betweenesscentrality,modularity_class,Count,Total,Degree Ratio,Subtopic,Main topic,Weight,Total Weight,Ratio Weight
Index,,,,,,,,,,,,,,,,
6985,49.0,14092434.0,Bibcode,0.515630,Q25754,STEM.Space,370573.961987,0.0,22.0,589.0,0.037351,Space,STEM,555.849083,7103.417204,0.078251
6987,49.0,14092434.0,Bibcode,0.370235,Q25754,STEM.Biology,370573.961987,0.0,23.0,589.0,0.039049,Biology,STEM,417.255277,7103.417204,0.058740
6986,49.0,14092434.0,Bibcode,0.484390,Q25754,STEM.Medicine_&_Health,370573.961987,0.0,16.0,589.0,0.027165,Medicine_&_Health,STEM,379.761799,7103.417204,0.053462
6988,49.0,14092434.0,Bibcode,0.287778,Q25754,STEM.Technology,370573.961987,0.0,20.0,589.0,0.033956,Technology,STEM,282.022284,7103.417204,0.039702
10865,5.0,40604331.0,Jydegaard Formation,0.996303,Q14975920,STEM.Earth_and_environment,6.500000,0.0,32.0,589.0,0.054329,Earth_and_environment,STEM,159.408445,7103.417204,0.022441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11623,3.0,46210705.0,Anthony Chickillo,0.000000,Q19877775,Geography.Regions.Americas.North_America,0.000000,40.0,8.0,27.0,0.296296,North_America,Geography,0.000000,104.401098,0.000000
12210,3.0,50189239.0,Sean Davis (American football),0.996633,Q24005664,Culture.Biography.Biography*,0.000000,40.0,6.0,27.0,0.222222,Biography*,Culture,0.000000,104.401098,0.000000
12212,3.0,50189239.0,Sean Davis (American football),0.000000,Q24005664,Geography.Regions.Americas.North_America,0.000000,40.0,8.0,27.0,0.296296,North_America,Geography,0.000000,104.401098,0.000000


,Main_Topics,Higher_Weighted_Article
0,Space (0.04) / Biology (0.04) / Medicine_&_Hea...,Bibcode (555)
1,Politics_and_government (0.08) / Earth_and_env...,2004 Indian Ocean earthquake and tsunami (278)
2,Politics_and_government (0.15) / Education (0....,Lindsey Graham (3410)
3,Sports (0.19) / Architecture (0.02) / Visual_a...,San Antonio Commanders (578)
4,Sports (0.3) / Media* (0.04) / Television (0.01),2015 Chicago Bears season (14012)
5,Military_and_warfare (0.11) / Transportation (...,Operation Ke (647)
6,Media* (0.2) / Films (0.09) / History (0.05),Cast a Dark Shadow (129)
7,Sports (0.2) / Society (0.03) / Media* (0.03),Split decision (827)
8,Media* (0.17) / Television (0.08) / Society (0...,65th Primetime Emmy Awards (6981)
9,Sports (0.29) / Media* (0.04) / Education (0.02),2018 Big Ten Conference football season (4797)


In [31]:
### Saving to nodes attributes into the graph ###

graph = nx.read_graphml(path+'graph.graphml')

df = pd.read_csv(path + 'pages_main_topic.csv', index_col = 'Index')
nodes = pd.DataFrame()

df.sort_values(by = ['Id', 'Weight'], inplace=True, ascending = [True, False])

list_id = list(dict.fromkeys(df['Id']))

# Keeping only the node with the higher Weight
for Id in list_id:
    id_max = df.loc[df['Id'] == Id, 'Weight'].idxmax()
    nodes = nodes.append(df.loc[df.index == id_max], ignore_index=True)

    
display(nodes)

    
add_graph_attribute(graph, nodes, 'Qid')
add_graph_attribute(graph, nodes, 'Topic')
add_graph_attribute(graph, nodes, 'Main topic')
add_graph_attribute(graph, nodes, 'Subtopic')
add_graph_attribute(graph, nodes, 'Weight')


nodes.to_csv(path + 'filled_nodes.csv', encoding='utf-8')

nx.write_graphml_lxml(graph, path + 'filled_graph.graphml')  



,Degree,Id,Label,Probability,Qid,Topic,betweenesscentrality,modularity_class,Count,Total,Degree Ratio,Subtopic,Main topic,Weight,Total Weight,Ratio Weight
0,3.0,876.0,Motor neuron disease,1.000010,Q3221083,STEM.Medicine_&_Health,1702.138702,11.0,1.0,1003.0,0.000997,Medicine_&_Health,STEM,3.000030,243315.894638,0.000012
1,12.0,2345.0,Archbishop of Canterbury,0.919653,Q29282,Culture.Philosophy_and_religion,126216.788382,29.0,21.0,176.0,0.119318,Philosophy_and_religion,Culture,231.752435,2408.185528,0.096235
2,1.0,2864.0,Archaeoastronomy,0.960371,Q463923,STEM.Space,0.000000,0.0,22.0,589.0,0.037351,Space,STEM,21.128166,7103.417204,0.002974
3,10.0,3335.0,Baltic Sea,0.985506,Q545,Geography.Regions.Europe.Europe*,107521.967845,0.0,42.0,589.0,0.071307,Europe*,Geography,0.000000,7103.417204,0.000000
4,4.0,3776.0,Bornholm,0.939923,Q769680,Geography.Regions.Europe.Europe*,4826.000000,0.0,42.0,589.0,0.071307,Europe*,Geography,0.000000,7103.417204,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3610,6.0,58599512.0,2018 Rakuten Japan Open Tennis Championships –...,0.997687,Q56843573,Culture.Sports,1.450000,22.0,44.0,167.0,0.263473,Sports,Culture,263.389332,7981.618867,0.032999
3611,11.0,58599535.0,2018 Sulawesi earthquake and tsunami,0.507822,Q56768333,History_and_Society.Politics_and_government,8266.992208,1.0,29.0,347.0,0.083573,Politics_and_government,History_and_Society,161.995173,3696.638661,0.043822
3612,1.0,58602901.0,Katherine Feinstein,0.201823,Q56798221,History_and_Society.Politics_and_government,0.000000,2.0,110.0,725.0,0.151724,Politics_and_government,History_and_Society,22.200554,65125.603728,0.000341
3613,2.0,58605337.0,Ana Maria Archila,0.206904,Q56810138,History_and_Society.History,0.000000,2.0,12.0,725.0,0.016552,History,History_and_Society,4.965697,65125.603728,0.000076
